In [ ]:
import sys
sys.path.append("../Pupillengröße/Skripte/")

import os
import numpy as np
from pyplr import utils
import pandas as pd
from classPLRfromGitHub import PLR
from pyplr import graphing
from pyplr import preproc
import matplotlib.pyplot as plt
from IPython.display import display

#import tkinter as tk
#from tkinter import simpledialog
import sys
import importlib
import checkdataPLR
import pup_config
import pup_util
import importlib
importlib.reload(pup_config)
importlib.reload(pup_util)

#from preprocessing import ProcessConfig
# import checkdataPLR


In [ ]:

# Define the subject_id
subject_id = 'PJ25_1_PLR3'

rec_dir=f"{pup_config.data_dir}/{subject_id[:4]}/{subject_id}"
utils.print_file_structure(rec_dir)
pup_util.get_condition(subject_id)
import lr_preprocessing
importlib.reload(lr_preprocessing)
config=lr_preprocessing.create_process_config(0,"diameter",subject_id,pup_config.data_dir)
config.out_dir='/home/marco/tmp/foo'
print(config)
lr_preprocessing.process(config,display)

In [ ]:
plr_all = [plr1, plr2, plr3, plr4]

for plr in plr_all:
    pyplr_result = get_pyplr_results(plr)
    pyplr_results["D1"].append(pyplr_result["D1"])
    pyplr_results["D2"].append(pyplr_result["D2"])
    pyplr_results["AMP"].append(pyplr_result["AMP"])
    pyplr_results["VCmax"].append(pyplr_result["VCmax"])
    pyplr_results["ACmax"].append(pyplr_result["ACmax"])
    pyplr_results["T1"].append(pyplr_result["T1"])
    pyplr_results["T2"].append(pyplr_result["T2"])
    pyplr_results["T3"].append(pyplr_result["T3"])
    pyplr_results["rel_AMP"].append(pyplr_result["rel_AMP"])
    pyplr_results["redil_75"].append(pyplr_result["redil_75"])
    pyplr_results["redil_50"].append(pyplr_result["redil_50"])
    pyplr_results["redil_25"].append(pyplr_result["redil_25"])




In [ ]:
df = pd.DataFrame(pyplr_results)
df["eye_id"] = "1"
df["method"] = "2d"


#always change the directory to what the sample gets saved 
df.to_csv(f"{pup_config.obj_dir}/{subject_id[:4]}/{subject_id}/pyplr_analysis/PLR_left_2d_results.csv", index=False)

print(df)
